In [1]:
import os, cv2
import numpy as np
from os import listdir
from os.path import isfile, join
from PIL import Image
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import shutil
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
import keras.optimizers
import random
import scipy
import math
import tensorflow as tf

In [2]:
imgFileDir = r'C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages\JPEGImages'
CImagesDir = r'C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages\CImages'
parentImgFileDir = r'C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages'
xmlFileDir = r'C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_XMLAnnotations\XMLAnnotations'


In [3]:
def extractClassLabels(xmlFileDir):
    os.chdir(xmlFileDir)
    xmlFileNames = [f for f in listdir(xmlFileDir) if isfile(join(xmlFileDir, f))]
    #xmlFileNames = ['3m1.xml','3m2.xml','3m3.xml','3m4.xml']
    classLabels = []
    imageNames = []
    count = 1
    for file in xmlFileNames:
        root = ET.parse(file).getroot()
        filename = root.find('filename').text
        action = root.find('object/action').text
        classLabels.append(action)
        imageNames.append(filename)
        print(str(count)+'/'+str(len(xmlFileNames)), end='\r')
        count +=1
    return np.array(classLabels), np.array(imageNames)

In [4]:
classList, allImageNames = extractClassLabels(xmlFileDir)


In [5]:
(classUnique,counts) = np.unique(classList, return_counts=True)

(_,inverse) = np.unique(classList, return_inverse=True)

y = to_categorical(inverse,len(classUnique))

In [6]:
batch_size = 32
img_height = 90
img_width = 90
actionImagesDir = r'C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages\ActionImages'
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    actionImagesDir,
    validation_split=0.2,
    label_mode = 'categorical',
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    actionImagesDir,
    validation_split=0.2,
    label_mode = 'categorical',
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 9532 files belonging to 40 classes.
Using 7626 files for training.
Found 9532 files belonging to 40 classes.
Using 1906 files for validation.


In [7]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [8]:
#AUTOTUNE = tf.data.experimental.AUTOTUNE
#train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
nrOfHiddenLayers = []
nrOfCNNLayers = []
nrOfHiddenLayerNeuron = []
nrOfCNNLayerNeuron = []
activationSelection = ['relu']

In [20]:
def define_model(nrOfClasses):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(90 ,90, 3)))
    model.add(MaxPooling2D((2, 2)))

    model.add(Dropout(0.50))

    model.add(Conv2D(64, (10, 10), activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.50))
    
    model.add(Conv2D(64, (30, 30), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Dropout(0.50))
    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(nrOfClasses, activation='softmax'))
    # compile model¨
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    #opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model
model = define_model(len(classUnique))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 88, 88, 32)        896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 44, 44, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 44, 44, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 35, 35, 64)        204864    
_________________________________________________________________
dropout_10 (Dropout)         (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 6, 6, 64)          3686464   
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 3, 3, 64)         

In [ ]:
model.fit(train_ds, epochs = 20, verbose=1)

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
239/239 [==============================] - 12s 52ms/step - loss: 3.7596 - accuracy: 0.0285
Epoch 2/20
239/239 [==============================] - 14s 58ms/step - loss: 3.6859 - accuracy: 0.0294
Epoch 3/20
239/239 [==============================] - 13s 56ms/step - loss: 3.6835 - accuracy: 0.0275
Epoch 4/20
239/239 [==============================] - 13s 53ms/step - lo

In [16]:
result = model.evaluate(test_ds, batch_size=32, verbose =1)
print(result)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
60/60 [==============================] - 2s 36ms/step - loss: 388.2030 - accuracy: 0.1264
[388.2030334472656, 0.1264428049325943]


In [12]:
print(np.__version__)

1.18.5
